In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
automation_file = "Resources/automation_data_by_state.csv"
automation_data_df = pd.read_csv(automation_file)

#take the hyphen out of the code
automation_data_df["SOC"] = automation_data_df['SOC'].astype(str)

automation_data_df["SOC"] = automation_data_df['SOC'].str.replace("-", '')

automation_data_df = automation_data_df.rename(columns={"SOC":"detail_id", "Occupation":"occupation",
                                                        "Probability":"probability"})

automation_data_df .set_index("detail_id", inplace=True)

automation_data_df.head()

,occupation,probability,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,...,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
detail_id,,,,,,,,,,,,,,,,,,,,,
111011,Chief Executives,0.015,1030,760,5750,2710,31150,880,1410,340,...,560,5460,5890,3650,280,6320,5910,980,3740,160
111021,General and Operations Managers,0.160,26930,6490,43300,20680,261780,41540,33280,4080,...,3730,44400,168610,36200,2760,52380,43760,10200,32350,4840
112011,Advertising and Promotions Managers,0.039,50,40,470,110,3760,480,300,0,...,0,670,1210,380,40,240,640,40,200,0
112021,Marketing Managers,0.014,530,200,4790,1090,33390,3060,4970,590,...,60,3400,9570,2320,380,3840,5830,260,2980,30
112022,Sales Managers,0.013,2510,400,10650,2650,69180,4570,7040,860,...,300,8890,22310,3360,480,5360,7390,600,5730,180


In [3]:
occupation_salary_file = "Resources/occupation_salary.csv"
occupation_salary_df = pd.read_csv(occupation_salary_file)
occupation_salary_df.set_index("OCC_CODE")


,OCC_TITLE,OCC_GROUP,TOT_EMP,EMP_PRSE,H_MEAN,A_MEAN,MEAN_PRSE,H_PCT10,H_PCT25,H_MEDIAN,H_PCT75,H_PCT90,A_PCT10,A_PCT25,A_MEDIAN,A_PCT75,A_PCT90,ANNUAL,HOURLY
OCC_CODE,,,,,,,,,,,,,,,,,,,
00-0000,All Occupations,total,"140,400,040",0.1,23.86,"49,630",0.1,9.27,11.60,17.81,28.92,45.45,"19,290","24,140","37,040","60,150","94,540",NaN,NaN
11-0000,Management Occupations,major,"7,090,790",0.2,56.74,"118,020",0.1,22.76,32.99,48.46,70.72,#,"47,330","68,630","100,790","147,090",#,NaN,NaN
11-1000,Top Executives,minor,"2,465,800",0.2,61.03,"126,950",0.2,20.58,31.45,49.19,78.35,#,"42,810","65,420","102,320","162,970",#,NaN,NaN
11-1010,Chief Executives,broad,"223,260",0.7,93.44,"194,350",0.4,33.55,54.86,87.12,#,#,"69,780","114,100","181,210",#,#,NaN,NaN
11-1011,Chief Executives,detailed,"223,260",0.7,93.44,"194,350",0.4,33.55,54.86,87.12,#,#,"69,780","114,100","181,210",#,#,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53-7111,Mine Shuttle Car Operators,detailed,"1,590",7.0,27.10,"56,370",1.3,21.21,23.98,27.14,29.91,32.73,"44,120","49,880","56,450","62,200","68,080",NaN,NaN
53-7120,"Tank Car, Truck, and Ship Loaders",broad,"10,920",13.1,19.04,"39,590",2.5,10.82,13.25,17.20,22.78,31.82,"22,510","27,570","35,770","47,380","66,190",NaN,NaN
53-7121,"Tank Car, Truck, and Ship Loaders",detailed,"10,920",13.1,19.04,"39,590",2.5,10.82,13.25,17.20,22.78,31.82,"22,510","27,570","35,770","47,380","66,190",NaN,NaN


In [4]:
#filter occupation dataframe to only major occupation group id and title
occupation_major_df = occupation_salary_df.loc[occupation_salary_df["OCC_GROUP"] == "major", ["OCC_CODE", "OCC_TITLE"]]

#take the hyphen out of the code
occupation_major_df["OCC_CODE"] = occupation_major_df['OCC_CODE'].astype(str)

occupation_major_df["OCC_CODE"] = occupation_major_df['OCC_CODE'].str.replace("-", '')

occupation_major_df = occupation_major_df.rename(columns={"OCC_CODE":"major_id"})

occupation_major_df.set_index("major_id", inplace=True)

occupation_major_df



,OCC_TITLE
major_id,
110000,Management Occupations
130000,Business and Financial Operations Occupations
150000,Computer and Mathematical Occupations
170000,Architecture and Engineering Occupations
190000,"Life, Physical, and Social Science Occupations"
210000,Community and Social Service Occupations
230000,Legal Occupations
250000,"Education, Training, and Library Occupations"
270000,"Arts, Design, Entertainment, Sports, and Media..."


In [5]:
#filter occupation dataframe to only minor occupation group id and title
occupation_minor_df = occupation_salary_df.loc[occupation_salary_df["OCC_GROUP"] == "minor", ["OCC_CODE", "OCC_TITLE"]]

#take the hyphen out of the code
occupation_minor_df["OCC_CODE"] = occupation_minor_df['OCC_CODE'].astype(str)

occupation_minor_df["OCC_CODE"] = occupation_minor_df['OCC_CODE'].str.replace("-", '')

occupation_minor_df = occupation_minor_df.rename(columns={"OCC_CODE":"minor_id"})

#add the associated broad OCC_CODE to each detail row to create link to other tables
#store last digit of the code in a variable
occupation_minor_df["major_id"] = occupation_minor_df['minor_id'].str[:2] + '0000'

#change id columns to integers
occupation_minor_df["major_id"] = occupation_minor_df['major_id'].astype(int)
occupation_minor_df["minor_id"] = occupation_minor_df["minor_id"].astype(int)

#set index to id column
occupation_minor_df.set_index("minor_id", inplace=True)

occupation_minor_df

,OCC_TITLE,major_id
minor_id,,
111000,Top Executives,110000
112000,"Advertising, Marketing, Promotions, Public Rel...",110000
113000,Operations Specialties Managers,110000
119000,Other Management Occupations,110000
131000,Business Operations Specialists,130000
...,...,...
533000,Motor Vehicle Operators,530000
534000,Rail Transportation Workers,530000
535000,Water Transportation Workers,530000


In [6]:
#filter occupation dataframe to only broad occupation group id and title
occupation_broad_df = occupation_salary_df.loc[occupation_salary_df["OCC_GROUP"] == "broad",
                                              ["OCC_CODE", "OCC_TITLE"]]

#take the hyphen out of the code
occupation_broad_df["OCC_CODE"] = occupation_broad_df['OCC_CODE'].astype(str)

occupation_broad_df["OCC_CODE"] = occupation_broad_df['OCC_CODE'].str.replace("-", '')

occupation_broad_df = occupation_broad_df.rename(columns={"OCC_CODE":"broad_id"})

#add the associated broad OCC_CODE to each detail row to create link to other tables
#store last digit of the code in a variable
occupation_broad_df["minor_id"] = occupation_broad_df['broad_id'].str[:4] + '00'

#change id columns to integers
occupation_broad_df["broad_id"] = occupation_broad_df['broad_id'].astype(int)
occupation_broad_df["minor_id"] = occupation_broad_df["minor_id"].astype(int)

#set index to id column
occupation_broad_df.set_index("broad_id", inplace=True)

occupation_broad_df

,OCC_TITLE,minor_id
broad_id,,
111010,Chief Executives,111000
111020,General and Operations Managers,111000
111030,Legislators,111000
112010,Advertising and Promotions Managers,112000
112020,Marketing and Sales Managers,112000
...,...,...
537070,Pumping Station Operators,537000
537080,Refuse and Recyclable Material Collectors,537000
537110,Mine Shuttle Car Operators,537100


In [7]:
#filter occupation dataframe to detail occupation data
occupation_detail_df = occupation_salary_df.loc[occupation_salary_df["OCC_GROUP"] == "detailed",
                                               ["OCC_CODE", "OCC_TITLE", "TOT_EMP", "A_MEAN", "A_MEDIAN", "H_MEAN", "H_MEDIAN"]]
occupation_detail_df.head(10)

occupation_detail_df.dtypes

OCC_CODE     object
OCC_TITLE    object
TOT_EMP      object
A_MEAN       object
A_MEDIAN     object
H_MEAN       object
H_MEDIAN     object
dtype: object

In [8]:
#take the hyphen out of the code
occupation_detail_df["OCC_CODE"] = occupation_detail_df['OCC_CODE'].astype(str)

occupation_detail_df["OCC_CODE"] = occupation_detail_df['OCC_CODE'].str.replace("-", '')


occupation_detail_df = occupation_detail_df.rename(columns={"OCC_CODE":"detail_id", "TOT_EMP":"total_emp",
                                                            "A_MEAN":"a_mean", "A_MEDIAN":"a_median", "H_MEAN": "h_mean",
                                                            "H_MEDIAN":"h_median"
                                                           })

#add the associated broad OCC_CODE to each detail row to create link to other tables
#store last digit of the code in a variable
occupation_detail_df["broad_id"] = occupation_detail_df['detail_id'].str[:5] + '0'

#change id columns to integers
occupation_detail_df["detail_id"] = occupation_detail_df['detail_id'].astype(int)
occupation_detail_df["broad_id"] = occupation_detail_df["broad_id"].astype(int)

#set index to id column
occupation_detail_df.set_index("detail_id", inplace=True)

occupation_detail_df.head(20)



,OCC_TITLE,total_emp,a_mean,a_median,h_mean,h_median,broad_id
detail_id,,,,,,,
111011,Chief Executives,"223,260","194,350","181,210",93.44,87.12,111010
111021,General and Operations Managers,"2,188,870","122,090","99,310",58.70,47.74,111020
111031,Legislators,"53,670","44,820","23,470",*,*,111030
112011,Advertising and Promotions Managers,"28,860","117,810","100,810",56.64,48.47,112010
112021,Marketing Managers,"205,900","144,140","131,180",69.30,63.07,112020
112022,Sales Managers,"365,230","135,090","117,960",64.95,56.71,112020
112031,Public Relations and Fundraising Managers,"63,970","123,360","107,320",59.31,51.59,112030
113011,Administrative Services Managers,"266,280","98,930","90,050",47.56,43.29,113010
113021,Computer and Information Systems Managers,"352,510","145,740","135,800",70.07,65.29,113020


In [9]:
#confirm datatypes of columns
occupation_detail_df.dtypes

OCC_TITLE    object
total_emp    object
a_mean       object
a_median     object
h_mean       object
h_median     object
broad_id      int32
dtype: object

In [10]:
connection_string = "postgres:postgres@localhost:5432/AutomationETL"
engine = create_engine(f'postgresql://{connection_string}')

In [11]:
# Confirm tables
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/e3q8)

In [12]:
#load tables
automation_data_df.to_sql(name='automation', con=engine, if_exists='append', index=False)

OperationalError: (psycopg2.OperationalError) FATAL:  password authentication failed for user "postgres"

(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
#load tables
occupation_major_df.to_sql(name='occupation_major', con=engine, if_exists='append', index=False)

In [ ]:
#load tables
occupation_mminor_df.to_sql(name='occupation_minor', con=engine, if_exists='append', index=False)

In [ ]:
#load tables
occupation_broad_df.to_sql(name='occupation_broad', con=engine, if_exists='append', index=False)

In [ ]:
#load tables
occupation_detail_df.to_sql(name='occupation_detail', con=engine, if_exists='append', index=False)